In [89]:
import numpy as np
import random
import time

Wskaźnik uwarunkowania macierzy

In [90]:
def norm(A):
    n = len(A)
    return max(sum(A[i][j] for j in range(n)) for i in range(n))

In [91]:
def conditioning_factor(A):
    A_inv = np.linalg.inv(A)
    return norm(A_inv) * norm(A)

Gaussian Elimination

![title](img_vsc/img01.png)

![title](img_vsc/img02.png)

In [92]:
def gaussian_elimination(A, B, data_type):
    
    n = A.shape[0]
    
    C = np.hstack((A, B.reshape(n, 1)))


    for i in range(n):
        for j in range(i + 1, n):
            ratio = C[j][i] / C[i][i]
            C[j] = C[j] - ratio * C[i]

    X = np.zeros(n, dtype=data_type)

    X[n - 1] = C[n - 1][n] / C[n - 1][n - 1] # We now have n + 1 columns

    for i  in range(n - 2 , -1, -1):
        X[i] = (C[i][n] - sum(X[i+1:n]*C[i][i+1:n])) / C[i][i]
        
    return X

Algorytm Thomasa dla macierzy trójdiagonalnej

In [141]:
def thomas_algorithm(A, B, C, D):
    
    N = len(B)
    
    C[0] /= B[0]
    D[0] /= B[0]
    
    for i in range(1, N - 1):
        
        common_coefficient = B[i] - A[i - 1] * C[i - 1]
        
        C[i] /= common_coefficient
        D[i] = (D[i] - A[i - 1] * D[i - 1])  / common_coefficient
        
    # Since loop goes only to n - 1, we need to calculate the last element of D
    D[N - 1] = (D[N - 1] - A[N - 2] * D[N - 2]) / (B[N - 1] - A[N - 2] * C[N - 2])
    
    # Backward substitution
    X = [0] * N
    X[-1] = D[-1]
    
    for i in range(N - 2, -1, -1):
        X[i] = D[i] - C[i] * X[i + 1]
        
    return X    

In [150]:
class GaussTest():
    
    def __init__(self, get_matrix_function):

        self.get_matrix = get_matrix_function

    def random_vector(self, n):
        
        V = []
        
        for _ in range(n):
            V.append( -1 if random.random() < 0.5 else 1)
            
        return V
    
        
    def solve(self, n, data_type):
        
        A = self.get_matrix(n)   
        X = self.random_vector(n)

        A = np.array(A, dtype=data_type).reshape(n, n)
        X = np.array(X, dtype=data_type).reshape(n)
        
        B = A @ X
        
        start = time.time()    

        X_approx = gaussian_elimination(A, B, data_type)

        end = time.time()
        
        if data_type == np.float32 or data_type == np.float64:
            return X, X_approx, end - start, conditioning_factor(A)
        else:
            return X, X_approx, end - start, None
        
    
    
    def solve_n(self, n_min, n_max, data_types):
        
        results = [[] for _ in range(len(data_types))]

        for n in range(n_min, n_max + 1):

            A = self.get_matrix(n)   
            X = self.random_vector(n)

            for i, data_type in enumerate(data_types):

                A = np.array(A, dtype=data_type).reshape(n, n)
                X = np.array(X, dtype=data_type).reshape(n)

                B = A @ X

                start = time.time()    

                X_approx = gaussian_elimination(A, B, data_type)

                end = time.time()

                if data_type == np.float32 or data_type == np.float64:
                    results[i].append([n, end - start, conditioning_factor(A), np.linalg.norm(X - X_approx)])
                else:
                    results[i].append([n, end - start, None, np.linalg.norm(X - X_approx)])
            
        return results
    
    @staticmethod
    def thomasify_matrix(A, data_type):

        a = []
        b = []
        c = []

        for i in range(len(A)):
            if i + 1 < len(A):
                a.append(A[i + 1][i])
                c.append(A[i][i + 1])
            b.append(A[i][i])

        a = np.array(a, dtype=data_type).reshape(len(a))
        b = np.array(b, dtype=data_type).reshape(len(b))
        c = np.array(c, dtype=data_type).reshape(len(c))

        return a, b, c


    def solve_n_thomas(self, n_min, n_max, data_types):

        results = [ [] for i in range(len(data_types))]

        for n in range(n_min, n_max + 1):

            A = self.get_matrix(n)
            X = self.random_vector(n)

            for i, data_type in enumerate(data_types):

                # Gaussian section

                A = np.array(A, dtype=data_type).reshape(n, n)
                X = np.array(X, dtype=data_type).reshape(n)

                B = A @ X

                gauss_start = time.time()    

                gauss_X_approx = gaussian_elimination(A, B, data_type)

                gauss_end = time.time()

                # Thomas section

                a, b, c = GaussTest.thomasify_matrix(A, data_type)

                thomas_start = time.time()

                thomas_X_approx = thomas_algorithm(a, b, c, B)

                thomas_end = time.time()

                if data_type == np.float32 or data_type == np.float64:
                    results[i].append([n, gauss_end - gauss_start, thomas_end - thomas_start, conditioning_factor(A), np.linalg.norm(X - gauss_X_approx), np.linalg.norm(X - thomas_X_approx)])
                else:
                    results[i].append([n, gauss_end - gauss_start, thomas_end - thomas_start, None, np.linalg.norm(X - gauss_X_approx), np.linalg.norm(X - thomas_X_approx)])
                
        return results

In [113]:
def zad1_matrix(n):
    
    A = [[0 for j in range(1, n + 1)] for i in range(1, n + 1)]
        
    for i in range(1, n + 1):
        for j in range(1, n + 1):
                
            if i == 1:
                A[i - 1][j - 1] = 1
            else:
                A[i - 1][j - 1] =  1 / (i + j - 1)
                    
    return A

Examples

In [100]:
zad1 = GaussTest(zad1_matrix)

X, X_approx, exec_time, cf = zad1.solve(5, np.float32)
print(X, X_approx, exec_time, cf)

[-1.  1. -1. -1. -1.] [-1.0000658   1.0007087  -1.0022193  -0.99733865 -1.0010849 ] 0.0015254020690917969 28023.636474609375


In [101]:
zad1 = GaussTest(zad1_matrix)

X, X_approx, exec_time, cf = zad1.solve(5, np.float64)
print(X, X_approx, exec_time, cf)

[ 1. -1. -1.  1. -1.] [ 1. -1. -1.  1. -1.] 0.00039887428283691406 28000.000000047658


Tests

In [122]:
zad1 = GaussTest(zad1_matrix)
zad1_results = zad1.solve_n(3, 300, [np.float32, np.float64])

[[], []]


In [124]:
zad1_results_float32 = zad1_results[0]

for n, exec_time, conditioning, error in zad1_results_float32:
    print(n, exec_time, conditioning, error)

3 0.0002129077911376953 216.00041484832764 0.0
4 0.000301361083984375 2880.032928466797 0.0003680778
5 0.00020456314086914062 28023.636474609375 0.0024549188
6 0.00030040740966796875 232606.4384765625 0.13677725
7 0.00031280517578125 2898755.69921875 3.3655167
8 0.0004317760467529297 1316895.181640625 5.8309016
9 0.0004227161407470703 14008956.3984375 13.752001
10 0.00048089027404785156 4207927.08984375 14.656978
11 0.0005645751953125 6356634.32421875 21.31492
12 0.0008318424224853516 2712885.1083984375 67.72306
13 0.0008120536804199219 2723394.9462890625 3400.5962
14 0.0008687973022460938 3962083.2719726562 34.590584
15 0.0009877681732177734 20024231.909179688 16.95359
16 0.0011305809020996094 19575512.140625 7.6201253
17 0.0012946128845214844 25471486.899414062 22.522089
18 0.0014464855194091797 16330873.974609375 102.19441
19 0.0015828609466552734 7791152.4873046875 29.804455
20 0.008159637451171875 5265721.1962890625 29.822836
21 0.0018715858459472656 9564600.394042969 22.520231
22

In [125]:
zad1_results_float64 = zad1_results[1]

for n, exec_time, conditioning, error in zad1_results_float64:
    print(n, exec_time, conditioning, error)

3 0.0001304149627685547 216.0003991134734 0.0
4 0.00013566017150878906 2880.032476674358 6.646518689688359e-15
5 0.0003032684326171875 28023.64869388366 3.3203860741371867e-12
6 0.00044655799865722656 232606.2172930967 1.8396801545635924e-12
7 0.00027370452880859375 2898707.638811715 5.255622297552604e-09
8 0.001331329345703125 1316884.9719799757 1.862509179215489e-09
9 0.00035119056701660156 14009484.881372094 1.5839291946008473e-08
10 0.0004382133483886719 4208044.049106911 6.452985080477134e-09
11 0.0006768703460693359 6356489.009672537 5.594401139179114e-09
12 0.0007882118225097656 2712841.002320409 1.055235563453851e-08
13 0.0006892681121826172 2723570.7333024815 1.704380753442368e-08
14 0.0008006095886230469 3962179.0572761893 1.4186203478936172e-08
15 0.0009014606475830078 20024346.98549062 3.314428079673662e-08
16 0.0010251998901367188 19576032.14162445 2.30640951719178e-08
17 0.0011820793151855469 25471425.697172284 4.408405378640109e-08
18 0.0013155937194824219 16331966.86200

In [126]:
def zad2_matrix(n):
    
    A = [[0 for j in range(1, n + 1)] for i in range(1, n + 1)]
    
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            
            if j >= i:
                A[i - 1][j - 1] = 2 * i / j
            else:
                A[i - 1][j - 1] = A[j - 1][i - 1]
                
    return A

Examples

In [127]:
zad2 = GaussTest(zad2_matrix)

X, X_approx, exec_time, cf = zad2.solve(5, np.float32)
print(X, X_approx, exec_time, cf)

[-1.  1.  1. -1. -1.] [-1.          1.0000001   0.99999994 -1.         -1.        ] 0.00045871734619140625 2.233333435654641


In [128]:
zad2 = GaussTest(zad2_matrix)

X, X_approx, exec_time, cf = zad2.solve(5, np.float64)
print(X, X_approx, cf)

[-1.  1.  1.  1.  1.] [-1.  1.  1.  1.  1.] 2.233333333333334


Tests

In [130]:
zad2 = GaussTest(zad2_matrix)
zad2_results = zad2.solve_n(3, 300, [np.float32, np.float64])

In [131]:
zad2_results_float32 = zad2_results[0]
for n, exec_time, conditioning, error in zad2_results_float32:
    print(n, exec_time, conditioning, error)

3 0.00025200843811035156 1.4444445007377202 0.0
4 0.0006120204925537109 1.8333334078391397 2.149076e-07
5 0.0004558563232421875 2.233333435654641 1.6858739e-07
6 0.00039649009704589844 2.6444445444477935 1.3734959e-06
7 0.0002655982971191406 3.031746150009214 9.348623e-07
8 0.00034737586975097656 3.4484128290935177 1.2921979e-06
9 0.00039958953857421875 3.849206492777855 3.5723028e-06
10 0.0005016326904296875 4.2492065205933525 3.4897998e-06
11 0.0005843639373779297 4.6594277916181355 1.9759636e-06
12 0.0007116794586181641 5.055219045933653 6.5456647e-06
13 0.0007686614990234375 5.465475483699896 3.705089e-06
14 0.0008769035339355469 5.868897985485124 9.723782e-06
15 0.001085519790649414 6.268898013300622 1.2691731e-05
16 0.001131296157836914 6.678405180923292 1.1428093e-05
17 0.001377105712890625 7.077618273425633 6.771885e-06
18 0.001439809799194336 7.485025688559575 9.406966e-06
19 0.0015850067138671875 7.889565429773881 1.5405927e-05
20 0.0017194747924804688 8.28956545758938 1.7198

In [132]:
zad2_results_float64 = zad2_results[1]
for n, exec_time, conditioning, error in zad2_results_float64:
    print(n, exec_time, conditioning, error)

3 7.462501525878906e-05 1.444444457689921 0.0
4 9.894371032714844e-05 1.8333333532015479 0.0
5 0.00014638900756835938 2.233333369096121 2.482534153247273e-16
6 0.00019288063049316406 2.644444465637208 6.377745716588144e-16
7 0.0002238750457763672 3.0317460596561405 2.1094237467877974e-15
8 0.00044226646423339844 3.44841272632281 1.0877919644084146e-15
9 0.0003523826599121094 3.849206378062566 1.6392246515987586e-15
10 0.0004284381866455078 4.2492063939571345 1.6203171603697468e-15
11 0.0005185604095458984 4.659427652756371 3.655315524298948e-15
12 0.0005962848663330078 5.05521889527639 1.1940963471123168e-14
13 0.0006909370422363281 5.465475320816039 6.130401827188391e-15
14 0.0007994174957275391 5.86889781057834 3.436315130343406e-15
15 0.0009169578552246094 6.268897826472909 1.5990680625176796e-14
16 0.0010488033294677734 6.678404981891315 1.4825531009620006e-14
17 0.001154184341430664 7.077618062496178 1.973981533156396e-14
18 0.0012886524200439453 7.485025465488432 7.84260790196416

In [133]:
def zad3a_matrix(n):

    k = 6
    m = 3

    A = [[0 for j in range(1, n + 1)] for i in range(1, n + 1)]

    for i in range(1, n + 1):
        for j in range(1, n + 1):

            if j == i:
                A[i - 1][j - 1] = k
            elif j == i + 1:
                A[i - 1][j - 1] = 1 / (i + m)
            elif i > 1 and j == i - 1:
                A[i - 1][j - 1] = k / (i + m + 1)

    return A

Examples

In [134]:
zad3a = GaussTest(zad3a_matrix)

X, X_approx, exec_time, cf = zad3a.solve(5, np.float16)
print(X, X_approx, exec_time, cf)

[ 1. -1. -1.  1.  1.] [ 1. -1. -1.  1.  1.] 0.00043272972106933594 None


In [135]:
zad3a = GaussTest(zad3a_matrix)

X, X_approx, exec_time, cf = zad3a.solve(5, np.float32)
print(X, X_approx, exec_time, cf)

[ 1.  1. -1.  1. -1.] [ 1.          0.99999994 -1.          1.         -1.0000001 ] 0.00047707557678222656 1.159485363403662


In [136]:
zad3a = GaussTest(zad3a_matrix)

X, X_approx, exec_time, cf = zad3a.solve(5, np.float64)
print(X, X_approx, exec_time, cf)

[ 1.  1. -1.  1. -1.] [ 1.  1. -1.  1. -1.] 0.0009186267852783203 1.1594853983864615


In [137]:
def zad3b_matrix(n):

    k = 6
    m = 3

    A = [[0 for j in range(1, n + 1)] for i in range(1, n + 1)]

    for i in range(1, n + 1):
        for j in range(1, n + 1):
            
            if j == i:
                A[i - 1][j - 1] = -m * i - k
            elif j == i + 1:
                A[i - 1][j - 1] = i
            elif i > 1 and j == i - 1:
                A[i - 1][j - 1] = m / i

    return A

Examples

In [138]:
zad3b = GaussTest(zad3b_matrix)

X, X_approx, exec_time, cf = zad3b.solve(5, np.float16)
print(X, X_approx, exec_time, cf)

[ 1. -1.  1.  1. -1.] [ 1. -1.  1.  1. -1.] 0.00033783912658691406 None


In [139]:
zad3b = GaussTest(zad3b_matrix)

X, X_approx, exec_time, cf= zad3b.solve(5, np.float32)
print(X, X_approx, exec_time, cf)

[-1. -1. -1.  1.  1.] [-1.         -1.         -1.          0.99999994  1.        ] 0.0003719329833984375 0.3970123413610054


In [140]:
zad3b = GaussTest(zad3b_matrix)

X, X_approx, exec_time, cf = zad3b.solve(5, np.float64)
print(X, X_approx, exec_time, cf)

[ 1. -1. -1.  1.  1.] [ 1. -1. -1.  1.  1.] 0.0009229183197021484 0.3970123323338485


Thomasify example

In [146]:
A = np.array([[10,2,0,0],
              [3,10,4,0],
              [0,1,7,5],
              [0,0,3,4]],dtype=float)   

a, b, c = GaussTest.thomasify_matrix(A, np.float64)
print(a)
print(b)
print(c)

[3. 1. 3.]
[10. 10.  7.  4.]
[2. 4. 5.]


In [152]:
zad3a = GaussTest(zad3a_matrix)
zad3a_results = zad3a.solve_n_thomas(3, 300, [np.float32, np.float64])

In [153]:
zad3_results_float32 = zad3a_results[0]

for n, gauss_exec_time, thomas_exec_time, conditioning, gauss_error, thomas_error in zad3_results_float32:
    print(n, gauss_exec_time, thomas_exec_time, conditioning, gauss_error, thomas_error)

3 0.0002162456512451172 1.8358230590820312e-05 1.159526202620794 1.3328004e-07 1.3328004e-07
4 0.0002722740173339844 2.002716064453125e-05 1.1594843539359265 1.6858739e-07 1.4600097e-07
5 0.00025534629821777344 4.410743713378906e-05 1.159485363403662 1.3328004e-07 1.3328004e-07
6 0.0011556148529052734 3.5762786865234375e-05 1.1594853421668128 1.6858739e-07 1.4600097e-07
7 0.0004467964172363281 3.075599670410156e-05 1.159485342565811 1.3328004e-07 1.7881393e-07
8 0.0005538463592529297 3.24249267578125e-05 1.159485342559107 1.7881393e-07 1.8848644e-07
9 0.0006363391876220703 3.552436828613281e-05 1.1594853425592093 1.7881393e-07 1.4600097e-07
10 0.0006630420684814453 4.9591064453125e-05 1.1594853425592078 2.149076e-07 2.4575624e-07
11 0.0006432533264160156 4.482269287109375e-05 1.1594853425592078 1.6858739e-07 2.0647654e-07
12 0.0006854534149169922 4.7206878662109375e-05 1.1594853425592078 1.3328004e-07 1.7881393e-07
13 0.001501321792602539 5.364418029785156e-05 1.1594853425592078 1.9768

In [154]:
zad3_results_float64 = zad3a_results[1]

for n, gauss_exec_time, thomas_exec_time, conditioning, gauss_error, thomas_error in zad3_results_float64:
    print(n, gauss_exec_time, thomas_exec_time, conditioning, gauss_error, thomas_error)

3 0.00017189979553222656 1.7404556274414062e-05 1.15952620005856 0.0 0.0
4 0.00021791458129882812 2.002716064453125e-05 1.1594843879478562 2.220446049250313e-16 2.220446049250313e-16
5 0.0007441043853759766 2.7418136596679688e-05 1.1594853988848859 0.0 2.220446049250313e-16
6 0.0003008842468261719 2.7418136596679688e-05 1.1594853775722198 3.1401849173675503e-16 0.0
7 0.000377655029296875 3.170967102050781e-05 1.1594853779708043 0.0 2.482534153247273e-16
8 0.00046634674072265625 3.170967102050781e-05 1.159485377964106 0.0 2.220446049250313e-16
9 0.0005471706390380859 4.506111145019531e-05 1.1594853779642083 0.0 2.220446049250313e-16
10 0.0007200241088867188 5.364418029785156e-05 1.1594853779642067 0.0 2.220446049250313e-16
11 0.0005104541778564453 5.14984130859375e-05 1.1594853779642067 2.220446049250313e-16 2.220446049250313e-16
12 0.0007686614990234375 5.078315734863281e-05 1.1594853779642067 1.1102230246251565e-16 2.7194799110210365e-16
13 0.0009622573852539062 5.435943603515625e-05 